In [ ]:
from google.cloud import bigquery
import pandas as pd
import functions_framework
from flask import jsonify
import logging

# Initialize BigQuery client
client = bigquery.Client()
gcloud functions deploy step-3-p-1 --runtime python311 --trigger-http --allow-unauthenticated --region=us-central1 --entry-point process_data
# Define source and target BigQuery tables
SOURCE_TABLE = 'trendsense.combined_data.step_3_predictive_1'
STOCK_HISTORY_TABLE = 'trendsense.stock_data.stock_data_history'
TARGET_TABLE = 'trendsense.combined_data.step_4_final'
REGRESSION_TABLE = 'trendsense.combined_data.step_4_test_train'

@functions_framework.http
def process_stock_data(request):
    try:
        logging.info("Starting data processing...")
        
        # First, get the regression coefficients
        regression_query = f"""
        SELECT *
        FROM `{REGRESSION_TABLE}`
        """
        regression_df = client.query(regression_query).to_dataframe()
        
        # Create a dictionary of regression coefficients for quick lookup
        regression_dict = {
            row['Ticker']: {
                'intercept': row['Intercept'],
                'ai_coef': row['AI_Coefficient'],
                'sentiment_coef': row['Sentiment_Coefficient'],
                'health_coef': row['Health_Coefficient']
            }
            for _, row in regression_df.iterrows()
        }

        # Modified query to use FULL OUTER JOIN
        query = f"""
      WITH stock_history AS (
  SELECT DISTINCT
    Ticker,
    DATE(Date) as Date,
    Close,
    Percent_Difference,
    LEAD(Percent_Difference) OVER (PARTITION BY Ticker ORDER BY Date) as Next_Day_Percent
  FROM `trendsense.stock_data.stock_data_history`
  WHERE ticker != 'GSAT'
),

predictive_data AS (
  SELECT
    ticker,
    DATE(date) as date,
    Stock_Category,
    Aggregated_Score,
    AI_Score,
    `Sentiment Score` as Sentiment_Score,
    Health_Score
  FROM `trendsense.combined_data.step_3_predictive_1`
  WHERE Ticker != 'GSAT'
)

SELECT 
  COALESCE(sh.Date, pd.date) as date,
  COALESCE(sh.Ticker, pd.ticker) as ticker,
  COALESCE(pd.Stock_Category, 'Unknown') as Stock_Category,
  pd.Aggregated_Score,
  pd.AI_Score,
  pd.Sentiment_Score as `Sentiment Score`,
  pd.Health_Score,
  sh.Close,
  sh.Percent_Difference as Avg_Daily_Percent_Difference,
  sh.Next_Day_Percent as Avg_Next_Daily_Percent_Difference
FROM (
  SELECT * FROM stock_history 
  UNION ALL
  SELECT DISTINCT ticker, date, NULL as Close, NULL as Percent_Difference, NULL as Next_Day_Percent 
  FROM predictive_data 
  WHERE date NOT IN (SELECT DISTINCT Date FROM stock_history)
) sh
FULL OUTER JOIN predictive_data pd
ON LOWER(sh.Ticker) = LOWER(pd.ticker)
AND sh.Date = pd.date
ORDER BY
  COALESCE(sh.Date, pd.date),
  COALESCE(sh.Ticker, pd.ticker)
        """
        
        # Rest of the function remains the same
        query_job = client.query(query)
        df = query_job.to_dataframe()

        filter_date = pd.to_datetime('2024-12-1')
        df['date'] = pd.to_datetime(df['date'])
        df = df[df['date'] >= filter_date]
        df = df.sort_values(['ticker', 'date'])
      

        
        # Additional filter for GSAT just in case
        df = df[df['ticker'] != 'GSAT']

        df_grouped = df.groupby(['ticker', 'date', 'Stock_Category']).agg({
            'Aggregated_Score': 'mean',
            'Close': 'last',
            'Avg_Daily_Percent_Difference': 'mean',
            'Avg_Next_Daily_Percent_Difference': 'mean',
            'AI_Score': 'mean',
            'Sentiment Score': 'mean',
            'Health_Score': 'mean'
        }).reset_index()

        # Updated column renaming
        df_grouped = df_grouped.rename(columns={
            'Aggregated_Score': 'TS_Score',
            'Avg_Daily_Percent_Difference': 'Price_Movement_Today',
            'Avg_Next_Daily_Percent_Difference': 'Price_Movement_Tomorrow',
            'AI_Score': 'AI_Score',
            'Sentiment Score': 'Sentiment_Score',
            'Health_Score': 'Health_Score'
        })
        # Convert 'date' in df_grouped to datetime (now it exists)
        df_grouped['date'] = pd.to_datetime(df_grouped['date'], errors='coerce')

        # Compute Week_of_Year with Saturday-Friday weeks
        df_grouped['Week_of_Year'] = (df_grouped['date'] - pd.Timedelta(days=-2)).dt.isocalendar().week
        # Calculate predicted next day percentage
        def predict_next_day(row):
            ticker_coef = regression_dict.get(row['ticker'])
            if ticker_coef:
                prediction = (
                    ticker_coef['intercept'] +
                    ticker_coef['ai_coef'] * row['AI_Score'] +
                    ticker_coef['sentiment_coef'] * row['Sentiment_Score'] +
                    ticker_coef['health_coef'] * row['Health_Score']
                )
                # Clip predictions to reasonable bounds
                return max(min(prediction, 0.05), -0.05)
            return None

        df_grouped['Predicted_Price_Movement'] = df_grouped.apply(predict_next_day, axis=1)

        # Continue with existing calculations
        df_grouped['Price_Movement_Today'] = df_grouped['Price_Movement_Today'].fillna(0)
        
        def calculate_4week_avg(group):
            """Calculate the average TS_Score for the last 4 weeks per ticker, excluding the current week."""
            group = group.sort_values(by="Week_of_Year")  # Ensure correct order

            def last_4_week_avg(row):
                last_4_weeks = group[
                    (group['Week_of_Year'] < row['Week_of_Year']) &  # Exclude current week
                    (group['Week_of_Year'] >= row['Week_of_Year'] - 4)  # Include only last 4 weeks
                ]
                return last_4_weeks['TS_Score'].mean() if not last_4_weeks.empty else None

            group['TS_Score_4Week'] = group.apply(last_4_week_avg, axis=1)

            return group

        # Apply function grouped by ticker
        df_grouped = df_grouped.groupby('ticker', group_keys=False).apply(calculate_4week_avg)

        # Rank stocks based on the 4-week average (higher average = better rank)
        df_grouped['TS_Rank_4Week'] = df_grouped.groupby('date')['TS_Score_4Week'].rank(
            method='min',
            ascending=False  # Higher scores get better ranks
        )

     
        # Calculate week-over-week change (last Friday vs previous Friday)
        def calculate_friday_change(group):
            # Get the last date in the data
            last_date = group['date'].max()
            
            # Find the last Friday
            days_to_friday = (last_date.dayofweek - 4) % 7
            last_friday = last_date - pd.Timedelta(days=days_to_friday)
            
            # Find the previous Friday
            prev_friday = last_friday - pd.Timedelta(weeks=1)
            
            # Get scores for these dates
            last_friday_score = group[group['date'] == last_friday]['TS_Score'].iloc[0] if len(group[group['date'] == last_friday]) > 0 else None
            prev_friday_score = group[group['date'] == prev_friday]['TS_Score'].iloc[0] if len(group[group['date'] == prev_friday]) > 0 else None
            
            if last_friday_score is not None and prev_friday_score is not None:
                pct_change = ((last_friday_score - prev_friday_score) / prev_friday_score) * 100
            else:
                pct_change = 0
                
            # Apply the change to all rows
            return pd.Series(pct_change, index=group.index)

        # Calculate and rank the Friday-to-Friday change
        df_grouped['TS_Friday_Change'] = df_grouped.groupby('ticker').apply(
            calculate_friday_change
        ).reset_index(level=0, drop=True)

        df_grouped['TS_Rank_Friday_Change'] = df_grouped.groupby('date')['TS_Friday_Change'].rank(
            method='min',
            ascending=False  # Higher change gets better ranks (lower numbers)
        )

        # Calculate 4-week composite score (average of both ranks)
        df_grouped['Composite_Rank_4Week'] = (df_grouped['TS_Rank_4Week'] + df_grouped['TS_Rank_Friday_Change']) / 2
        
        #####################################################
        # Get the most recent Composite_Rank_4Week for each ticker in each week
        latest_rank = (
            df_grouped.groupby(['Week_of_Year', 'ticker'])['Composite_Rank_4Week']
            .last()
            .reset_index()
        )

        # Calculate the sum of Price_Movement_Today for each ticker within each week
        weekly_ticker_sums = (
            df_grouped.groupby(['Week_of_Year', 'ticker'])['Price_Movement_Today']
            .sum()
            .reset_index()
        )

        # Merge to get Composite Rank per ticker per week
        weekly_ticker_analysis = weekly_ticker_sums.merge(
            latest_rank, 
            on=['Week_of_Year', 'ticker']
        )

        # Get the top 10 tickers based on Composite_Rank_4Week and their individual sums
        weekly_top_10_sums = (
            weekly_ticker_analysis.groupby('Week_of_Year')
            .apply(lambda x: x.nsmallest(10, 'Composite_Rank_4Week'))
            .reset_index(drop=True)
            .rename(columns={'Price_Movement_Today': 'Weekly_Ticker_Movement'})
        )

        # Calculate the average movement for top 10 tickers per week
        weekly_averages = (
            weekly_top_10_sums.groupby('Week_of_Year')['Weekly_Ticker_Movement']
            .mean()
            .reset_index()
            .rename(columns={'Weekly_Ticker_Movement': 'Weekly_Ticker_Avg_Movement'})
        )

        # Merge the individual ticker movements and weekly averages back to the main dataframe
        df_grouped = df_grouped.merge(
            weekly_top_10_sums[['Week_of_Year', 'ticker', 'Weekly_Ticker_Movement']],
            on=['Week_of_Year', 'ticker'],
            how='left'
        )

        df_grouped = df_grouped.merge(
            weekly_averages,
            on='Week_of_Year',
            how='left'
        )

        # Fill NaN values with 0
        df_grouped['Weekly_Ticker_Movement'] = df_grouped['Weekly_Ticker_Movement'].fillna(0)
        df_grouped['Weekly_Ticker_Avg_Movement'] = df_grouped['Weekly_Ticker_Avg_Movement'].fillna(0)

        # Compute weekly cumulative sum for 2025 weeks based on average movement
        weekly_2025_returns = (
            df_grouped[df_grouped['date'] >= '2025-01-01']
            .groupby('Week_of_Year')['Weekly_Ticker_Avg_Movement']
            .first()
            .reset_index()
        )

        # Calculate cumulative sum of the averages
        weekly_2025_returns['Weekly_Ticker_Cumulative'] = weekly_2025_returns['Weekly_Ticker_Avg_Movement'].cumsum()

        # Merge back to main dataframe
        df_grouped = df_grouped.merge(
            weekly_2025_returns[['Week_of_Year', 'Weekly_Ticker_Cumulative']],
            on='Week_of_Year',
            how='left'
        )

        # Fill NaN values with 0 for pre-2025 dates
        df_grouped['Weekly_Ticker_Cumulative'] = df_grouped.apply(
            lambda row: row['Weekly_Ticker_Cumulative'] if row['date'].year >= 2025 else 0,
            axis=1
        )
        ########################################################
        df_grouped['TS_Score_7'] = df_grouped.groupby('ticker')['TS_Score'].transform(
            lambda x: x.rolling(window=7, min_periods=1).mean()
        )

        df_grouped['TS_Rank_7'] = df_grouped.groupby('date')['TS_Score_7'].rank(
            method='min',
            ascending=False
        )

        df_grouped['TS_Change'] = df_grouped.groupby('ticker')['TS_Score'].pct_change() * 100
        df_grouped['TS_Rank_Change'] = df_grouped.groupby('date')['TS_Change'].rank(
            method='min',
            ascending=False
        )

        df_grouped['Composite_Rank'] = (df_grouped['TS_Rank_7'] + df_grouped['TS_Rank_Change']) / 2
        
        # Calculate daily top 10 averages with new column names
       
       
        daily_top_10_avg_next = (
            df_grouped.groupby('date')
            .apply(lambda x: x.nsmallest(10, 'Composite_Rank')['Price_Movement_Tomorrow'].mean())
            .reset_index()
            .rename(columns={0: 'Top_10_Composite_Price_Movement_Tomorrow'})
        )
        
        # Fill NaN values with 0 for Top_10_Composite_Price_Movement_Tomorrow
        daily_top_10_avg_next['Top_10_Composite_Price_Movement_Tomorrow'] = (
            daily_top_10_avg_next['Top_10_Composite_Price_Movement_Tomorrow'].fillna(0)
        )

        daily_top_10_avg_today = (
            df_grouped.groupby('date')
            .apply(lambda x: x.nsmallest(10, 'Composite_Rank')['Price_Movement_Today'].mean())
            .reset_index()
            .rename(columns={0: 'Top_10_Composite_Price_Movement_Today'})
        )

        daily_top_10_predicted = (
            df_grouped.groupby('date')
            .apply(lambda x: x.nsmallest(10, 'Composite_Rank')['Predicted_Price_Movement'].mean())
            .reset_index()
            .rename(columns={0: 'Top_10_Predicted_Price_Movement'})
        )

        # Merge the daily averages back
        df_grouped = df_grouped.merge(daily_top_10_avg_next, on='date', how='left')
        df_grouped = df_grouped.merge(daily_top_10_avg_today, on='date', how='left')
        df_grouped = df_grouped.merge(daily_top_10_predicted, on='date', how='left')

        # Calculate cumulative sums for 2025
        df_2025 = df_grouped[df_grouped['date'] >= '2025-01-01'].copy()

        daily_cumulative_next = (
            df_2025.groupby('date')['Top_10_Composite_Price_Movement_Tomorrow']
            .mean()
            .cumsum()
            .reset_index()
            .rename(columns={'Top_10_Composite_Price_Movement_Tomorrow': 'Top_10_YTD_Cumulative_Tomorrow'})
        )

        daily_cumulative_today = (
            df_2025.groupby('date')['Top_10_Composite_Price_Movement_Today']
            .mean()
            .cumsum()
            .reset_index()
            .rename(columns={'Top_10_Composite_Price_Movement_Today': 'Top_10_YTD_Cumulative_Today'})
        )

        daily_cumulative_predicted = (
            df_2025.groupby('date')['Top_10_Predicted_Price_Movement']
            .mean()
            .cumsum()
            .reset_index()
            .rename(columns={'Top_10_Predicted_Price_Movement': 'Top_10_YTD_Cumulative_Predicted'})
        )

        # Merge cumulative sums
        df_grouped = df_grouped.merge(daily_cumulative_next, on='date', how='left')
        df_grouped = df_grouped.merge(daily_cumulative_today, on='date', how='left')
        df_grouped = df_grouped.merge(daily_cumulative_predicted, on='date', how='left')

        # Fill missing cumulative scores
        df_grouped['Top_10_YTD_Cumulative_Tomorrow'].fillna(0, inplace=True)
        df_grouped['Top_10_YTD_Cumulative_Today'].fillna(0, inplace=True)
        df_grouped['Top_10_YTD_Cumulative_Predicted'].fillna(0, inplace=True)

        # Final sort
        df_grouped = df_grouped.sort_values(['date'])

        # Save to BigQuery (overwrite existing table)
        job_config = bigquery.LoadJobConfig(
            write_disposition=bigquery.WriteDisposition.WRITE_TRUNCATE,
            autodetect=True
        )

        job = client.load_table_from_dataframe(df_grouped, TARGET_TABLE, job_config=job_config)
        job.result()

        return jsonify({
            "message": f"Data successfully overwritten in {TARGET_TABLE}",
            "row_count": len(df_grouped)
        })

    except Exception as e:
        logging.error(f"Error in processing: {str(e)}")
        return jsonify({"error": str(e)}), 500


  Attempting uninstall: yfinance
    Found existing installation: yfinance 0.2.49
    Uninstalling yfinance-0.2.49:
      Successfully uninstalled yfinance-0.2.49



[notice] A new release of pip is available: 24.2 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [ ]:
import yfinance as yf
import pandas as pd
import logging
from datetime import datetime, timedelta

# Configure logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

# Define stock tickers
TICKERS = [
    'AAPL', 'GOOGL', 'MSFT', 'ASTS', 'PTON', 'GSAT', 'PLTR', 'SMR', 'ACHR',
    'BWXT', 'ARBK', 'AMD', 'NVDA', 'GME', 'MU', 'TSLA', 'NFLX', 'ZG',
    'AVGO', 'SMCI', 'GLW', 'HAL', 'LMT', 'AMZN', 'CRM', 'NOW', 'CHTR', 'TDS', 'META', 'RGTI', 'QUBT',
    'LX', 'OKLO', 'PSIX', 'QFIN', 'RTX', 'TWLO'
]

# Market Index Tickers
INDEX_TICKERS = ["^IXIC", "^GSPC"]  # NASDAQ and S&P 500

def extract_stock_close():
    """Fetch stock data and save it locally as a CSV file."""
    try:
        today = datetime.today().date()
        start_date = today - timedelta(days=3)
        
        logger.info(f"Fetching stock data from {start_date} to {today}")
        logger.info(f"Fetching NASDAQ (^IXIC) and S&P 500 (^GSPC) data from {start_date} to {today}")

        # Fetch stock data
        try:
            stock_data = yf.download(TICKERS, start=start_date, end=today, group_by='ticker', threads=5)
        except Exception as download_error:
            logger.error(f"Failed to download stock data: {download_error}")
            return

        # Fetch NASDAQ and S&P 500 data
        try:
            index_data = yf.download(INDEX_TICKERS, start=start_date, end=today, group_by='ticker')
        except Exception as index_error:
            logger.error(f"Failed to download index data: {index_error}")
            return

        formatted_data = []

        # Process normal tickers using Yahoo Finance dates
        for ticker in TICKERS:
            try:
                if ticker in stock_data.columns.get_level_values(0):
                    ticker_data = stock_data[ticker].reset_index()  # Convert index to column

                    for _, row in ticker_data.iterrows():
                        if pd.notna(row['Close']):
                            previous_close = ticker_data['Close'].shift(1).iloc[_] if _ > 0 else None
                            percent_difference = ((row['Close'] - previous_close) / previous_close) if previous_close else None

                            formatted_data.append({
                                "Date": row["Date"].strftime('%Y-%m-%d'),
                                "Ticker": ticker,
                                "Close": row['Close'],
                                "Volume": row['Volume'],
                                "High": row['High'],
                                "Low": row['Low'],
                                "Open": row['Open'],
                                "Percent_Difference": percent_difference
                            })
            except Exception as ticker_error:
                logger.error(f"Error processing {ticker}: {ticker_error}")
                continue

        # Process NASDAQ and S&P 500 Data
        for ticker in INDEX_TICKERS:
            if ticker in index_data.columns.get_level_values(0):
                index_df = index_data[ticker].reset_index()  # Convert index to column
                
                for _, row in index_df.iterrows():
                    try:
                        previous_close = index_df["Close"].shift(1).iloc[_] if _ > 0 else None
                        percent_difference = ((row["Close"] - previous_close) / previous_close) if previous_close else None

                        formatted_data.append({
                            "Date": row["Date"].strftime('%Y-%m-%d'),
                            "Ticker": ticker,
                            "Close": row["Close"],
                            "Volume": row["Volume"] if "Volume" in index_df.columns else None,
                            "High": row["High"] if "High" in index_df.columns else None,
                            "Low": row["Low"] if "Low" in index_df.columns else None,
                            "Open": row["Open"] if "Open" in index_df.columns else None,
                            "Percent_Difference": percent_difference
                        })
                    except Exception as index_error:
                        logger.error(f"Error processing {ticker}: {index_error}")
                        continue

        # Convert to DataFrame
        reformatted_data = pd.DataFrame(formatted_data)

        if reformatted_data.empty:
            logger.warning(f"No valid stock data available for {today}")
            return

        # Save to CSV file locally
        file_path = f"stock_data_{today}.csv"
        reformatted_data.to_csv(file_path, index=False)
        
        logger.info(f"Stock data saved locally as {file_path}")

    except Exception as general_error:
        logger.error(f"Unexpected error in extract_stock_close: {general_error}")

if __name__ == "__main__":
    extract_stock_close()



  # Add 4-week Friday-to-Friday average calculation
        df_grouped['is_friday'] = df_grouped['date'].dt.day_name() == 'Friday'

  def calculate_friday_4week_avg(group):
            friday_data = group[group['is_friday']].copy()
            if friday_data.empty:
                return pd.Series(index=group.index, dtype='float64')
            
            friday_averages = friday_data['TS_Score'].rolling(window=4, min_periods=1).mean()
            friday_data['TS_Score_4Week'] = friday_averages
            
            result = friday_data['TS_Score_4Week'].reindex(group.index).ffill()
            return result

        df_grouped['TS_Score_4Week'] = df_grouped.groupby('ticker').apply(
            calculate_friday_4week_avg
        ).reset_index(level=0, drop=True)

        # Drop the helper column as it's no longer needed
        df_grouped = df_grouped.drop('is_friday', axis=1)
        
         # Rank stocks based on 4-week average (higher average = better rank)
        df_grouped['TS_Rank_4Week'] = df_grouped.groupby('date')['TS_Score_4Week'].rank(
            method='min',
            ascending=False  # Higher scores get better ranks (lower numbers)
        )










INFO:__main__:Fetching stock data from 2025-02-10 to 2025-02-13
INFO:__main__:Fetching NASDAQ (^IXIC) and S&P 500 (^GSPC) data from 2025-02-10 to 2025-02-13


[*********************100%***********************]  37 of 37 completed
[*********************100%***********************]  2 of 2 completed
INFO:__main__:Stock data saved locally as stock_data_2025-02-13.csv


In [ ]:
import yfinance as yf
import pandas as pd
from datetime import datetime, timedelta
from google.cloud import bigquery
import logging

# Configure logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

# Define BigQuery dataset and table
PROJECT_ID = "trendsense"
DATASET_ID = "stock_data"
TABLE_ID = "stock_data_history"

# Define the list of stock tickers
TICKERS = [
    'AAPL', 'GOOGL', 'MSFT', 'ASTS', 'PTON', 'GSAT', 'PLTR', 'SMR', 'ACHR',
    'BWXT', 'ARBK', 'AMD', 'NVDA', 'GME', 'MU', 'TSLA', 'NFLX', 'ZG',
    'AVGO', 'SMCI', 'GLW', 'HAL', 'LMT', 'AMZN', 'CRM', 'NOW', 'CHTR', 'TDS', 'META', 'RGTI','QUBT',
    'LX', 'OKLO', 'PSIX', 'QFIN', 'RTX', 'TWLO'
]

def extract_stock_close(request):
    """Cloud Function to fetch current day stock data and save to BigQuery."""
    try:
        # Define today's date and previous business day
        today = datetime.today()
        
        # Adjust for weekends and market holidays
        start_date = today - timedelta(days=3)
        end_date = today
        
        logger.info(f"Fetching stock data from {start_date} to {end_date}")
        
        # Fetch stock data using a date range to ensure data availability
        try:
            stock_data = yf.download(TICKERS, start=start_date, end=end_date, group_by='ticker', threads=True)
        except Exception as download_error:
            logger.error(f"Failed to download stock data: {download_error}")
            return f"Failed to download stock data: {download_error}"

        # Check if data was returned
        if stock_data.empty:
            logger.warning(f"No data available for date range {start_date} to {end_date}")
            return f"No data available for date range {start_date} to {end_date}"

        # Create an empty list to store reformatted data
        formatted_data = []

        # Process each ticker to extract relevant information
        for ticker in TICKERS:
            try:
                if ticker in stock_data.columns.get_level_values(0):  # Ensure ticker exists in data
                    # Select the most recent day's data
                    ticker_data = stock_data[ticker].iloc[-1]
                    
                    # Ensure we have valid data for the current day
                    if pd.notna(ticker_data['Close']):
                        # Calculate percent difference from the previous close
                        try:
                            previous_close = stock_data[ticker].iloc[-2]['Close']
                            current_close = ticker_data['Close']
                            percent_difference = ((current_close - previous_close) / previous_close)
                        except (IndexError, TypeError):
                            previous_close = None
                            percent_difference = None

                        # Append today's data
                        formatted_data.append({
                            "Date": today.strftime('%Y-%m-%d'),
                            "Ticker": ticker,
                            "Close": ticker_data['Close'],
                            "Volume": ticker_data['Volume'],
                            "High": ticker_data['High'],
                            "Low": ticker_data['Low'],
                            "Open": ticker_data['Open'],
                            "Percent_Difference": percent_difference
                        })
            except Exception as ticker_error:
                logger.error(f"Error processing {ticker}: {ticker_error}")
                continue

        # Convert the list of dictionaries to a DataFrame
        reformatted_data = pd.DataFrame(formatted_data)

        # Check if reformatted data is empty
        if reformatted_data.empty:
            logger.warning(f"No valid stock data available for {today}")
            return f"No valid stock data available for {today}"

        # Save to BigQuery
        try:
            client = bigquery.Client(project=PROJECT_ID)
            table_ref = f"{PROJECT_ID}.{DATASET_ID}.{TABLE_ID}"
            
            # Define job configuration
            job_config = bigquery.LoadJobConfig(
                write_disposition=bigquery.WriteDisposition.WRITE_APPEND,  # Append data if table exists
                autodetect=True  # Automatically detect schema
            )

            # Load data to BigQuery
            job = client.load_table_from_dataframe(
                reformatted_data,
                table_ref,
                job_config=job_config
            )
            
            # Wait for job to complete and log any errors
            job.result()
            
            logger.info(f"Stock data for {today} successfully saved to {table_ref}")
            return f"Stock data for {today} successfully saved to {table_ref}"

        except Exception as bigquery_error:
            logger.error(f"BigQuery upload failed: {bigquery_error}")
            return f"BigQuery upload failed: {bigquery_error}"

    except Exception as general_error:
        logger.error(f"Unexpected error in extract_stock_close: {general_error}")
        return f"Unexpected error: {general_error}"

# Note: If this is a Google Cloud Function, you might need to add a trigger
# such as a HTTP trigger or a scheduled cloud function trigger









In [1]:
!pip install yahooquery


   ---------------------------------------- 0.0/3.8 MB ? eta -:--:--
   ------------------- -------------------- 1.8/3.8 MB 11.2 MB/s eta 0:00:01
   -------------------------------------- - 3.7/3.8 MB 11.5 MB/s eta 0:00:01
   ---------------------------------------- 3.8/3.8 MB 9.0 MB/s eta 0:00:00
  Attempting uninstall: lxml
    Found existing installation: lxml 5.3.0
    Uninstalling lxml-5.3.0:
      Successfully uninstalled lxml-5.3.0



[notice] A new release of pip is available: 24.2 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [51]:
import yfinance as yf
import pandas as pd
from datetime import datetime, timedelta
import logging

# Configure logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

# Define stock tickers
TICKERS = [
    'AAPL', 'GOOGL', 'MSFT', 'ASTS', 'PTON', 'GSAT', 'PLTR', 'SMR', 'ACHR',
    'BWXT', 'ARBK', 'AMD', 'NVDA', 'GME', 'MU', 'TSLA', 'NFLX', 'ZG',
    'AVGO', 'SMCI', 'GLW', 'HAL', 'LMT', 'AMZN', 'CRM', 'NOW', 'CHTR', 'TDS', 'META', 'RGTI', 'QUBT',
    'LX', 'OKLO', 'PSIX', 'QFIN', 'RTX', 'TWLO'
]

# NASDAQ Composite Index Ticker
NASDAQ_TICKER = "^IXIC"

def extract_stock_close():
    """Fetch stock data for normal tickers (past 3 days) and NASDAQ (^IXIC) (since Dec 1, 2024)."""
    try:
        today = datetime.today()
        start_date = today - timedelta(days=3)  # Normal tickers (last 3 days)
        nasdaq_start_date = datetime(2024, 12, 1)  # Fixed year for NASDAQ

        logger.info(f"Fetching stock data from {start_date} to {today}")
        logger.info(f"Fetching NASDAQ (^IXIC) data from {nasdaq_start_date} to {today}")

        # Fetch stock data for normal tickers
        try:
            stock_data = yf.download(TICKERS, start=start_date, end=today, group_by='ticker', threads=5)
            logger.info(f"Stock data downloaded: {len(stock_data)} rows")
        except Exception as download_error:
            logger.error(f"Failed to download stock data: {download_error}")
            return
        
        # Fetch NASDAQ Composite Index (^IXIC) data
        try:
            nasdaq_data = yf.download(NASDAQ_TICKER, start=nasdaq_start_date, end=today)
            logger.info(f"NASDAQ data downloaded: {nasdaq_data.shape} rows and columns")
        except Exception as nasdaq_error:
            logger.error(f"Failed to download NASDAQ data: {nasdaq_error}")
            return

        # Debug: Print column names before fixing
        print("\nNASDAQ Column Names BEFORE FIX:", nasdaq_data.columns)

        # 🛠 FIX: Flatten NASDAQ MultiIndex Columns
        if isinstance(nasdaq_data.columns, pd.MultiIndex):
            nasdaq_data.columns = [col[0] for col in nasdaq_data.columns]
        nasdaq_data = nasdaq_data.reset_index()  # Convert Date index into a column

        # Detect correct column names dynamically
        column_map = {
            "Close": None,
            "Open": None,
            "High": None,
            "Low": None,
            "Volume": None
        }

        for col in nasdaq_data.columns:
            for key in column_map.keys():
                if key in col:
                    column_map[key] = col

        # Rename columns using detected names
        nasdaq_data = nasdaq_data.rename(columns=column_map)

        # Debug: Print final column names after renaming
        print("\nNASDAQ Column Names FINAL:", nasdaq_data.columns)

        formatted_data = []

        # Process normal tickers
        for ticker in TICKERS:
            try:
                if ticker in stock_data.columns.get_level_values(0):
                    ticker_data = stock_data[ticker].iloc[-1]  # Get latest data
                    
                    if pd.notna(ticker_data['Close']):
                        # Find previous close safely
                        previous_close = stock_data[ticker]['Close'].shift(1).iloc[-1]
                        percent_difference = None
                        
                        if pd.notna(previous_close):
                            percent_difference = ((ticker_data['Close'] - previous_close) / previous_close)

                        formatted_data.append({
                            "Date": today.strftime('%Y-%m-%d'),
                            "Ticker": ticker,
                            "Close": ticker_data['Close'],
                            "Volume": ticker_data['Volume'],
                            "High": ticker_data['High'],
                            "Low": ticker_data['Low'],
                            "Open": ticker_data['Open'],
                            "Percent_Difference": percent_difference
                        })
            except Exception as ticker_error:
                logger.error(f"Error processing {ticker}: {ticker_error}")
                continue

        # Process NASDAQ Data
        for _, row in nasdaq_data.iterrows():
            try:
                # Compute percent difference safely
                previous_close = nasdaq_data["Close"].shift(1).iloc[-1] if len(nasdaq_data) > 1 else None
                percent_difference = None

                if pd.notna(previous_close):
                    percent_difference = ((row["Close"] - previous_close) / previous_close)

                formatted_data.append({
                    "Date": row["Date"].strftime('%Y-%m-%d'),
                    "Ticker": NASDAQ_TICKER,
                    "Close": row["Close"],
                    "Volume": row["Volume"] if "Volume" in nasdaq_data.columns else None,
                    "High": row["High"] if "High" in nasdaq_data.columns else None,
                    "Low": row["Low"] if "Low" in nasdaq_data.columns else None,
                    "Open": row["Open"] if "Open" in nasdaq_data.columns else None,
                    "Percent_Difference": percent_difference
                })
            except Exception as nasdaq_error:
                logger.error(f"Error processing NASDAQ (^IXIC): {nasdaq_error}")
                continue

        # Convert to DataFrame
        reformatted_data = pd.DataFrame(formatted_data)

        # Save to CSV for debugging
        reformatted_data.to_csv("output.csv", index=False)
        logger.info("Saved output.csv for verification.")

    except Exception as general_error:
        logger.error(f"Unexpected error: {general_error}")

# Run the function locally
extract_stock_close()







INFO:__main__:Fetching stock data from 2025-01-31 17:02:50.095980 to 2025-02-03 17:02:50.095980
INFO:__main__:Fetching NASDAQ (^IXIC) data from 2024-12-01 00:00:00 to 2025-02-03 17:02:50.095980
[************          24%                       ]  9 of 37 completedWARNING:urllib3.connectionpool:Connection pool is full, discarding connection: query2.finance.yahoo.com. Connection pool size: 10
[*************         27%                       ]  10 of 37 completedWARNING:urllib3.connectionpool:Connection pool is full, discarding connection: query2.finance.yahoo.com. Connection pool size: 10
[***************       32%                       ]  12 of 37 completedWARNING:urllib3.connectionpool:Connection pool is full, discarding connection: query2.finance.yahoo.com. Connection pool size: 10
[******************    38%                       ]  14 of 37 completedWARNING:urllib3.connectionpool:Connection pool is full, discarding connection: query2.finance.yahoo.com. Connection pool size: 10
[******


NASDAQ Column Names BEFORE FIX: MultiIndex([('Adj Close', '^IXIC'),
            (    'Close', '^IXIC'),
            (     'High', '^IXIC'),
            (      'Low', '^IXIC'),
            (     'Open', '^IXIC'),
            (   'Volume', '^IXIC')],
           names=['Price', 'Ticker'])

NASDAQ Column Names FINAL: Index(['Date', 'Adj Close', 'Close', 'High', 'Low', 'Open', 'Volume'], dtype='object')


: 

In [43]:
# Yahoo Extract with date restriction 

import nltk
import os

# Explicitly set the nltk_data path
nltk_data_path = r"C:\Users\BryceDaniel\OneDrive - Lincoln Telephone Company\MSBA\GitHub\TrendSense\Market News\Market_News_Yahoo_Extract_Function\nltk_data"
nltk.data.path.append(nltk_data_path)

# Ensure 'punkt' is downloaded into the correct folder
nltk.download('punkt', download_dir=nltk_data_path)

import yfinance as yf
import pandas as pd
from datetime import datetime, timedelta
from textblob import TextBlob


def calculate_sentiment(text):
    try:
        analysis = TextBlob(text)
        return analysis.sentiment.polarity
    except Exception as e:
        print(f"[ERROR] Sentiment analysis failed: {e}")
        return 0

def label_sentiment(score):
    if score > 0.35:
        return "Bullish"
    elif 0.15 < score <= 0.35:
        return "Somewhat-Bullish"
    elif -0.15 <= score <= 0.15:
        return "Neutral"
    elif -0.35 <= score < -0.15:
        return "Somewhat-Bearish"
    else:
        return "Bearish"

def get_market_news(tickers, days_back=2):
    all_news = []
    today = datetime.now().date()
    cutoff_date = today - timedelta(days=days_back)

    for ticker in tickers:
        stock = yf.Ticker(ticker)
        try:
            news = stock.news
            for item in news:
                try:
                    # Extract publish timestamp and date
                    publish_timestamp = item.get('providerPublishTime', 0)
                    publish_date = datetime.fromtimestamp(publish_timestamp).date()

                    # Only process news within the desired date range
                    if publish_date >= cutoff_date:
                        title = item.get('title', '')
                        sentiment_score = calculate_sentiment(title)
                        sentiment_label = label_sentiment(sentiment_score)

                        news_item = {
                            'ticker': ticker,
                            'title': title,
                            'summary': title,  # Replicate title in the summary column
                            'publisher': item.get('publisher', ''),
                            'link': item.get('link', ''),
                            'publish_date': datetime.fromtimestamp(publish_timestamp),
                            'type': item.get('type', ''),
                            'related_tickers': ', '.join(item.get('relatedTickers', [])),
                            'source': 'yahoo',
                            'overall_sentiment_score': sentiment_score,
                            'overall_sentiment_label': sentiment_label,
                        }
                        all_news.append(news_item)
                except Exception as news_item_error:
                    print(f"[ERROR] Error processing news item: {news_item_error}")
        except Exception as e:
            print(f"[ERROR] Error retrieving news for {ticker}: {str(e)}")
    return pd.DataFrame(all_news)

def save_to_csv(df, filename):
    try:
        df.to_csv(filename, index=False)
        print(f"[INFO] Data successfully saved to {filename}")
    except Exception as e:
        print(f"[ERROR] Failed to save data to CSV: {e}")

def fetch_and_save_market_news():
    indices = ['^IXIC', '^DJI', '^RUT', '^GSPC']
    market_news = get_market_news(tickers=indices)
    if not market_news.empty:
        market_news['category'] = 'General'

    tech_stocks = [
        'AAPL', 'GOOGL', 'MSFT', 'ASTS', 'PTON', 'GSAT', 'PLTR', 'SMR', 'ACHR',
        'BWXT', 'ARBK', 'AMD', 'NVDA', 'BTC', 'GME', 'MU', 'TSLA', 'NFLX', 'ZG',
        'AVGO', 'SMCI', 'GLW', 'HAL', 'LMT', 'AMZN', 'CRM', 'NOW', 'CHTR', 'TDS', 'META','RGTI','QUBT',
        'LX', 'OKLO', 'PSIX', 'QFIN', 'RTX', 'TWLO'
    ]
    tech_news = get_market_news(tickers=tech_stocks)
    if not tech_news.empty:
        tech_news['category'] = 'Tech'

    combined_news = pd.concat([market_news, tech_news], ignore_index=True)

    if not combined_news.empty:
        save_to_csv(combined_news, "market_news.csv")
    else:
        print("[INFO] No news data to save.")

if __name__ == "__main__":
    fetch_and_save_market_news()


[nltk_data] Downloading package punkt to C:\Users\BryceDaniel\OneDrive
[nltk_data]     - Lincoln Telephone
[nltk_data]     Company\MSBA\GitHub\TrendSense\Market
[nltk_data]     News\Market_News_Yahoo_Extract_Function\nltk_data...
[nltk_data]   Unzipping tokenizers\punkt.zip.


[INFO] Data successfully saved to market_news.csv


In [1]:
!pip install --upgrade yfinance





  Attempting uninstall: yfinance
    Found existing installation: yfinance 0.2.50
    Uninstalling yfinance-0.2.50:
      Successfully uninstalled yfinance-0.2.50



[notice] A new release of pip is available: 24.2 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [36]:
import requests
import yfinance as yf

class StockPriceTargetRetriever:
    def __init__(self, api_key=None):
        """
        Initialize the Stock Price Target Retriever
        
        :param api_key: API key for paid services (optional)
        """
        self.api_key = api_key
    
    def get_yahoo_finance_target(self, symbol):
        """
        Retrieve price targets and recommendations from Yahoo Finance
        
        :param symbol: Stock ticker symbol
        :return: Dictionary with recommendations and price targets
        """
        try:
            # Fetch the stock information
            stock = yf.Ticker(symbol)
            
            # Fetch analyst recommendations
            recommendations = stock.recommendations
            
            # Fetch analyst price targets
            info = stock.info
            
            # Extract price target information from stock info
            price_targets = {
                'current_price': info.get('currentPrice'),
                'target_high_price': info.get('targetHighPrice'),
                'target_low_price': info.get('targetLowPrice'),
                'target_mean_price': info.get('targetMeanPrice'),
                'target_median_price': info.get('targetMedianPrice')
            }
            
            return {
                'recommendations': recommendations,
                'price_targets': price_targets
            }
        except Exception as e:
            print(f"Error fetching Yahoo Finance data: {e}")
            return None
    
    def get_alpha_vantage_overview(self, symbol):
        """
        Retrieve stock overview from Alpha Vantage
        
        :param symbol: Stock ticker symbol
        :return: Dictionary of stock overview data
        """
        if not self.api_key:
            raise ValueError("Alpha Vantage requires an API key")
        
        url = f'https://www.alphavantage.co/query?function=OVERVIEW&symbol={symbol}&apikey={self.api_key}'
        try:
            response = requests.get(url)
            response.raise_for_status()
            return response.json()
        except requests.RequestException as e:
            print(f"Error fetching data from Alpha Vantage: {e}")
            return None
    
    def get_financial_modeling_prep_target(self, symbol):
        """
        Retrieve price targets from Financial Modeling Prep
        
        :param symbol: Stock ticker symbol
        :return: List of price target data
        """
        if not self.api_key:
            raise ValueError("Financial Modeling Prep requires an API key")
        
        url = f'https://financialmodelingprep.com/api/v3/price-target?symbol={symbol}&apikey={self.api_key}'
        try:
            response = requests.get(url)
            response.raise_for_status()
            return response.json()
        except requests.RequestException as e:
            print(f"Error fetching data from Financial Modeling Prep: {e}")
            return None

def main():
    # Initialize the retriever
    retriever = StockPriceTargetRetriever()
    
    # Retrieve price targets for Apple (AAPL)
    symbol = 'ASTS'
    
    # Yahoo Finance (completely free)
    yahoo_targets = retriever.get_yahoo_finance_target(symbol)
    
    # Print results with error handling
    if yahoo_targets:
        print("Yahoo Finance Targets:")
        print("Recommendations:")
        print(yahoo_targets.get('recommendations', 'No recommendations available'))
        print("\nPrice Targets:")
        price_targets = yahoo_targets.get('price_targets', {})
        for key, value in price_targets.items():
            print(f"{key.replace('_', ' ').title()}: {value}")
    else:
        print("Failed to retrieve stock information.")

if __name__ == '__main__':
    main()

# Important Notes:
# 1. This script requires yfinance library
# 2. Install dependencies: pip install yfinance requests
# 3. Be aware of potential rate limits or changes in Yahoo Finance's structure

# Troubleshooting:
# - Ensure you have the latest version of yfinance
# - Some stock symbols might not have complete information
# - Network connectivity can affect data retrieval
   

Yahoo Finance Targets:
Recommendations:
  period  strongBuy  buy  hold  sell  strongSell
0     0m          2    3     0     0           0
1    -1m          2    3     0     0           0
2    -2m          2    3     0     0           0
3    -3m          2    3     0     0           0

Price Targets:
Current Price: 25.645
Target High Price: 53.0
Target Low Price: 15.0
Target Mean Price: 35.94
Target Median Price: 36.0


Email sent successfully.


In [25]:
import yfinance as yf
import pandas as pd
from datetime import datetime, timedelta
from newspaper import Article
import nltk

# Ensure the required NLTK data is downloaded
nltk.download('punkt')

def fetch_article_summary(link):
    """
    Fetch and summarize the article content from a URL.
    """
    try:
        # Use Newspaper3k with headers
        article = Article(link)
        article.download()
        article.parse()
        article.nlp()
        return article.summary
    except Exception as e:
        print(f"[ERROR] Newspaper3k failed for {link}: {str(e)}. Falling back to BeautifulSoup.")

        # Fallback to BeautifulSoup
        try:
            response = requests.get(link, headers={'User-Agent': 'Mozilla/5.0'})
            response.raise_for_status()
            soup = BeautifulSoup(response.content, "html.parser")
            paragraphs = soup.find_all("p")
            content = " ".join([p.get_text() for p in paragraphs])
            return content[:500] + "..." if len(content) > 500 else content
        except Exception as bs_error:
            print(f"[ERROR] BeautifulSoup also failed for {link}: {str(bs_error)}")
            return "No summary available."




def get_market_news(tickers):
    """
    Fetch market news for the current day, capturing all available fields and generating summaries.
    Only processes news items with 'type' set to 'story'.
    """
    all_news = []
    today = datetime.now().date()
    one_day_ago = today - timedelta(days=1)

    for ticker in tickers:
        stock = yf.Ticker(ticker)

        try:
            news = stock.news
            for item in news:
                try:
                    publish_timestamp = item.get('providerPublishTime', 0)
                    publish_date = datetime.fromtimestamp(publish_timestamp).date()

                    # Filter news to include only today's and yesterday's articles
                    if publish_date >= one_day_ago:
                        # Only summarize articles with type 'story'
                        if item.get('type', '').lower() == 'story':
                            link = item.get('link', '')
                            summary = fetch_article_summary(link) if link else "No summary available."

                            news_item = {
                                'ticker': ticker,
                                'title': item.get('title', ''),
                                'publisher': item.get('publisher', ''),
                                'link': link,
                                'publish_date': datetime.fromtimestamp(publish_timestamp),
                                'summary': summary,  # Include the generated summary
                                'type': item.get('type', ''),  # Original type from Yahoo API
                                'related_tickers': ', '.join(item.get('relatedTickers', [])),  # Comma-separated related tickers
                            }
                            all_news.append(news_item)
                        else:
                            print(f"[INFO] Skipping non-story type: {item.get('type', '')}")
                except Exception as news_item_error:
                    print(f"[ERROR] Error processing news item: {news_item_error}")

        except Exception as e:
            print(f"[ERROR] Error retrieving news for {ticker}: {str(e)}")

    print(f"[INFO] Fetched {len(all_news)} news articles.")
    return pd.DataFrame(all_news)


def save_to_csv(df, output_dir="market_news"):
    """
    Save processed news data to a CSV file locally.
    """
    try:
        if df.empty:
            print("[INFO] No news data to save.")
            return None

        os.makedirs(output_dir, exist_ok=True)
        filename = f"market_news_{datetime.now().strftime('%Y%m%d_%H%M%S')}.csv"
        filepath = os.path.join(output_dir, filename)
        df.to_csv(filepath, index=False, encoding="utf-8")
        print(f"[INFO] News data saved locally at: {filepath}")
        return filepath
    except Exception as e:
        print(f"[ERROR] Failed to save CSV: {str(e)}")
        return None


def main():
    """
    Main function for fetching and saving market news locally.
    """
    try:
        # Fetch general market news
        indices = ['^IXIC', '^DJI', '^RUT', '^GSPC']
        market_news = get_market_news(tickers=indices)
        if not market_news.empty:
            market_news['category'] = 'General'  # Add category for general market

        # Fetch tech stock news
        tech_stocks = ['AAPL', 'GOOGL', 'MSFT']
        tech_news = get_market_news(tickers=tech_stocks)
        if not tech_news.empty:
            tech_news['category'] = 'Tech'  # Add category for tech stocks

        # Combine news
        combined_news = pd.concat([market_news, tech_news], ignore_index=True)

        # Save to CSV locally
        if not combined_news.empty:
            save_to_csv(combined_news)
        else:
            print("[INFO] No news data to save.")
    except Exception as e:
        print(f"[ERROR] Error in main function: {e}")


if __name__ == "__main__":
    main()

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\BryceDaniel\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


[INFO] Skipping non-story type: VIDEO
[INFO] Skipping non-story type: VIDEO
[ERROR] Newspaper3k failed for https://finance.yahoo.com/news/p-500-closes-record-high-213627406.html: 
**********************************************************************
  Resource punkt_tab not found.
  Please use the NLTK Downloader to obtain the resource:

  >>> import nltk
  >>> nltk.download('punkt_tab')
  
  For more information see: https://www.nltk.org/data.html

  Attempted to load tokenizers/punkt_tab/english/

  Searched in:
    - 'C:\\Users\\BryceDaniel/nltk_data'
    - 'c:\\Users\\BryceDaniel\\AppData\\Local\\Programs\\Python\\Python312\\nltk_data'
    - 'c:\\Users\\BryceDaniel\\AppData\\Local\\Programs\\Python\\Python312\\share\\nltk_data'
    - 'c:\\Users\\BryceDaniel\\AppData\\Local\\Programs\\Python\\Python312\\lib\\nltk_data'
    - 'C:\\Users\\BryceDaniel\\AppData\\Roaming\\nltk_data'
    - 'C:\\nltk_data'
    - 'D:\\nltk_data'
    - 'E:\\nltk_data'
    - 'c:\\Users\\BryceDaniel\\OneDrive